# IMPORT Section

In [3]:
try:
    import google.colab
    from cultural_classification_nlp.src.utils import dataset_parser
    print("You are running on Colab!")
    colab = True

except:
    print("You are running locally!")
    colab = False

import os


You are running on Colab!


# Clone of the repository
Cloning the repository if you are on Colab

In [2]:
if colab:
    !git clone "https://github.com/cybernetic-m/cultural_classification_nlp.git"

Cloning into 'cultural_classification_nlp'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 44 (delta 16), reused 33 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 211.57 KiB | 2.09 MiB/s, done.
Resolving deltas: 100% (16/16), done.


# Dataset

In [4]:
DATASET_PATH = "cultural_classification_nlp/train_set.tsv"

df = dataset_parser(DATASET_PATH)

The dataset is:

                                    item                                 name  \
0    http://www.wikidata.org/entity/Q306                     Sebastián Piñera   
1  http://www.wikidata.org/entity/Q12735                   John Amos Comenius   
2   http://www.wikidata.org/entity/Q1752                             Macrinus   
3   http://www.wikidata.org/entity/Q1639                         Lamine Diack   
4   http://www.wikidata.org/entity/Q9588                        Richard Nixon   
5   http://www.wikidata.org/entity/Q1625                      Sigfrid Edström   
6   http://www.wikidata.org/entity/Q1628  David Cecil, 6th Marquess of Exeter   
7  http://www.wikidata.org/entity/Q12706                          Maxim Gorky   
8    http://www.wikidata.org/entity/Q207                       George W. Bush   
9  http://www.wikidata.org/entity/Q12702                         Robert Cecil   

                                         description    type  category  \
0    Chilean entr